In [ ]:
from sympy import symbols, Function, Eq, lambdify
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import numpy as np

x, y1, y2, y3, y4, y5, y6, y7, y8, y9 = symbols('x y1 y2 y3 y4 y5 y6 y7 y8 y9')
Y = [f'y{i}' for i in range(1, 10)]
P = [0.82645, 3.0312, 26.844, 0.22028, 22.807, 30.438, 5.7443, 9.6769, 1809.6]
S = [0.56596, 0.06717, 1.151, 0.03181, 0.97227, 0.037697, 0.024872, 0.60515, 258.52]
columna_K = [1.6273, 58.209, 65.218, 14.988, 17.705, 272.67, 11.981, 56.946, 4.9391, 34.691, 4.779, 18.819, 61.511, 1.3297, 8.4207, 11.72, 14.013, 11.867, 9.5381, 1.7108, 4.6155, 23.779, 39.778, 3.0976, 1.5401, 9.762, 13.822, 1.3686e6, 50.903, 150.6, 9.3893, 134.06, 15.778, 2.4818, 8.9587, 40.718, 14.545, 376.12, 351.21, 27.191, 133.91, 9.7889, 14.225]
columna_Yi = ['y2', 'y3', 'y5', 'y7', 'y8', 'y6', 'y3', 'y5', 'y7', 'y1', 'y8', 'y6', 'y5', 'y4', 'y8', 'y6', 'y5', 'y3', 'y2', 'y7', 'y9', 'y8', 'y1', 'y6', 'y5', 'y3', 'y2', 'y1', 'y7', 'y8', 'y5', 'y3', 'y2', 'y1', 'y8', 'y6', 'y7', 'y5', 'y3', 'y2', 'y6', 'y1', 'y5']
columna_act_repre = [1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1]
columna_Yj = ['y1', 'y1', 'y1', 'y1', 'y1', 'y1', 'y2', 'y2', 'y2', 'y2', 'y2', 'y2', 'y3', 'y3', 'y3', 'y3', 'y4', 'y5', 'y5', 'y5', 'y5', 'y5', 'y5', 'y5', 'y6', 'y6', 'y6', 'y6', 'y6', 'y6', 'y7', 'y7', 'y7', 'y7', 'y7', 'y7', 'y8', 'y8', 'y8', 'y8', 'y8', 'y8', 'y9']
columna_N = [1.6354, 1.6833, 1.8982, 2.3757, 2.2303, 1.5392, 2.674, 3.5452, 1.4626, 1.3153, 1.8499, 3.0102, 4.9999, 1.1567, 1.8104, 2.117, 2.049, 1.8588, 2.2855, 3.3169, 2.3944, 2.9639, 1.7128, 2.0704, 60.901, 1.3191, 4.0316, 1.8207, 1.9367, 1.3081, 1.9267, 1.9118, 2.3397, 1.9367, 1.7976, 1.3055, 2.6053, 1.1819, 1.8328, 1.9916, 2.9068, 2.0699, 2.041]
columna_b = [82.645]*6 + [3.0312]*6 + [26.844]*4 + [22.028]*1 + [22.807]*7 + [30.438]*6 + [5.7443]*6 + [9.6769]*6 + [1809.6]*1

# Crear un diccionario para agrupar los elementos según su correspondencia con columna_Yj
grupo_elementos = {}
for yj, elemento_K, elemento_Yi, elemento_act_repre, elemento_N, elemento_b in zip(columna_Yj, columna_K, columna_Yi, columna_act_repre, columna_N, columna_b):
    if yj not in grupo_elementos:
        grupo_elementos[yj] = {'K': [], 'Yi': [], 'act_repre': [], 'N': [], 'b': [], 'Yj': []}
    grupo_elementos[yj]['K'].append(elemento_K)
    grupo_elementos[yj]['Yi'].append(elemento_Yi)
    grupo_elementos[yj]['act_repre'].append(elemento_act_repre)
    grupo_elementos[yj]['N'].append(elemento_N)
    grupo_elementos[yj]['b'].append(elemento_b)
    grupo_elementos[yj]['Yj'].append(yj)

# Crear sublistas para cada grupo de elementos idénticos en columna_Yj
interacción_y = []
for yj, elementos in grupo_elementos.items():
    interacción_y.append([elementos['K'], elementos['Yi'], elementos['act_repre'], elementos['N'], elementos['b'], elementos['Yj']])

# Crear una lista para almacenar los objetos Eq de las ecuaciones diferenciales
eqs = []

# Crear una lista para almacenar las soluciones de las ecuaciones diferenciales
soluciones_ecuaciones = []

for j in range(len(interacción_y)):

    K = interacción_y[j][0]
    y = interacción_y[j][1]
    switch_act_repre = interacción_y[j][2]
    N = interacción_y[j][3]
    beta =interacción_y[j][4]
    yj = interacción_y[j][5]
    indices = [i for i in range(len(y))]

    sum_numerador = 0
    sum_denominador = 0

    for i in range(len(y)):
        y[i] = symbols(y[i])
        sum_numerador += beta[i] * (y[i] / K[i]) ** (N[i] if switch_act_repre[i] == 1 else 0)
        sum_denominador += beta[i] * (y[i] / K[i]) ** N[i]

    yj[0] = symbols(yj[0])
    eq = Eq(Function(yj[0])(x).diff(x), (P[j]*(sum_numerador / (1 + sum_denominador)))-S[j]*yj[0])
    eqs.append(eq)

for eq in eqs:
    print(eq)
# Convertir las ecuaciones diferenciales simbólicas en funciones numéricas
funciones_numéricas = [lambdify((x, y1, y2, y3, y4, y5, y6, y7, y8, y9), eq.rhs, np) for eq in eqs]

# Resto del código...

# Definir las condiciones iniciales y los valores de x para evaluar las ecuaciones diferenciales
y0 = [10.4, 10.5, 9.5, 10.6, 10.5, 7.4, 9.2, 10.5, 9.5]  # Condiciones iniciales para y1, y2, ..., y9
x_vals = np.linspace(0, 60, 100)  # Valores de x para evaluar las ecuaciones

# Resolver las ecuaciones diferenciales numéricamente
soluciones = odeint(lambda y, x: [f(x, *y) for f in funciones_numéricas], y0, x_vals)

# Obtener las soluciones para cada variable y separarlas en una lista
soluciones_y = [soluciones[:, i] for i in range(9)]

# Imprimir las soluciones
for i, solucion_y in enumerate(soluciones_y):
    print(f"y{i+1}: {solucion_y}")

# Graficar las soluciones en imágenes separadas
for i, solucion_y in enumerate(soluciones_y):
    plt.figure()  # Crear una nueva figura
    plt.plot(x_vals, solucion_y)  # Graficar la solución correspondiente
    plt.xlabel('x')
    plt.ylabel(f'y{i+1}')
    plt.title(f'Solución para y{i+1}')
    plt.grid(True)
    plt.show()  # Mostrar la figura actual